In [2]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter
from typing import List
from langchain.schema import Document

def load_hr_document(file_path: str) -> List[Document]:
    """
    HR 정책 문서를 TextLoader + MarkdownHeaderTextSplitter로 로드하는 함수
    
    Args:
        file_path (str): 마크다운 파일 경로
    
    Returns:
        List[Document]: 분할된 문서 청크들
    """
    
    # 1. TextLoader로 문서 로드
    loader = TextLoader(file_path, encoding="utf-8")
    documents = loader.load()
    
    # 2. Document 객체에서 텍스트 내용 추출
    document_text = documents[0].page_content
    
    # 3. HR 문서 구조에 맞는 헤더 정의
    headers_to_split_on = [
        ("#", "문서제목"),          # # 가이다 플레이 스튜디오(GPS) 직원 복지제도 종합 안내서
        ("##", "정책대분류"),       # ## 1. 휴가 및 휴직 제도
        ("###", "정책세부항목"),    # ### 1.1 연차휴가
        # ("####", "세부절차"),       # #### **신청 절차**
    ]
    
    # 4. MarkdownHeaderTextSplitter로 구조적 분할
    markdown_splitter = MarkdownHeaderTextSplitter(
        headers_to_split_on=headers_to_split_on,
        strip_headers=False  # 헤더 정보 유지 (컨텍스트에 중요)
    )
    
    # 5. 문자열을 split_text에 전달 (Document가 아닌 str)
    md_header_splits = markdown_splitter.split_text(document_text)
    
    # 6. 긴 섹션을 위한 추가 텍스트 분할
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,        # HR Q&A에 적합한 크기
        chunk_overlap=200,      # 충분한 컨텍스트 오버랩
        separators=["\n\n", "\n", ". ", "? ", "! ", " ", ""]
    )
    
    # 7. 최종 분할 적용
    final_splits = text_splitter.split_documents(md_header_splits)
    
    return final_splits

In [ ]:
# FAISS 거리 점수 해석 가이드
0.0 - 0.3   # 🟢 매우 유사 (거의 동일한 내용)
0.3 - 0.7   # 🟡 적당히 유사 (관련성 높음)  
0.7 - 1.2   # 🟠 약간 유사 (일부 관련성)
1.2 - 2.0   # 🔴 덜 유사 (관련성 낮음)
2.0+        # ⚫ 거의 무관 (다른 주제)

# HR 챗봇 추천 설정
conservative_threshold = 0.8   # 엄격 (높은 정확도)
balanced_threshold = 1.0       # 균형 (권장)  
lenient_threshold = 1.3        # 관대 (높은 재현율)

In [7]:
import os
from dotenv import load_dotenv
from typing import List, Dict, Tuple, Any
import time
from dataclasses import dataclass

# LangChain imports
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document

@dataclass
class TestQuery:
    """테스트 쿼리와 예상 정답 정보"""
    query: str
    expected_sections: List[str]  # 예상되는 정답 섹션 키워드들
    description: str

@dataclass 
class SearchResult:
    """검색 결과 정보"""
    content: str
    score: float
    metadata: Dict

@dataclass
class EvaluationResult:
    """평가 결과"""
    query: str
    precision_at_k: float
    hit_rate: bool
    retrieved_sections: List[str]
    scores: List[float]
    is_relevant: List[bool]

class HRSimilaritySearchEvaluator:
    """HR 문서 유사도 검색 평가기"""
    
    def __init__(self, file_path: str, k: int = 3, score_threshold: float = 0.75):
        """
        Args:
            file_path: HR 정책 문서 경로
            k: 상위 k개 결과 반환
            score_threshold: 유사도 임계값
        """
        load_dotenv()  # .env 파일 로드
        
        self.file_path = file_path
        self.k = k
        self.score_threshold = score_threshold
        self.embeddings = None
        self.vectorstore = None
        self.test_queries = self._create_test_queries()
        
    def _create_test_queries(self) -> List[TestQuery]:
        """HR 문서 기반 테스트 쿼리 생성"""
        return [
            TestQuery(
                query="연차휴가는 몇 일인가요?",
                expected_sections=["연차휴가", "15일", "기본"],
                description="기본 연차 일수 문의"
            ),
            TestQuery(
                query="병가 신청은 어떻게 하나요?",
                expected_sections=["병가", "신청 절차", "전자결재"],
                description="병가 신청 절차 문의"
            ),
            TestQuery(
                query="복지포인트는 얼마나 받나요?",
                expected_sections=["복지포인트", "300만원", "연간"],
                description="복지포인트 지급액 문의"
            ),
            TestQuery(
                query="교육비 지원 한도는 얼마인가요?",
                expected_sections=["교육비", "50만원", "연간"],
                description="교육비 지원 한도 문의"
            ),
            TestQuery(
                query="개인 장비 구입 지원은 어떻게 되나요?",
                expected_sections=["개인장비", "200만원", "2년"],
                description="개인장비 지원 제도 문의"
            ),
            TestQuery(
                query="건강검진은 누가 받을 수 있나요?",
                expected_sections=["건강검진", "본인", "배우자", "부모"],
                description="건강검진 대상 문의"
            ),
            TestQuery(
                query="육아 지원금은 얼마인가요?",
                expected_sections=["육아 지원금", "20만원", "만 8세"],
                description="육아 지원금 금액 문의"
            ),
            TestQuery(
                query="동아리 활동비는 어떻게 지원되나요?",
                expected_sections=["동아리", "2만원", "1인", "1개월"],
                description="동아리 활동비 지원 문의"
            ),
            TestQuery(
                query="가족돌봄휴가는 유급인가요?",
                expected_sections=["가족돌봄휴가", "무급", "연 10일"],
                description="가족돌봄휴가 급여 문의"
            ),
            TestQuery(
                query="태아 검진 휴가는 몇 번 사용할 수 있나요?",
                expected_sections=["태아 검진", "유급", "반차"],
                description="태아 검진 휴가 횟수 문의"
            ),
            TestQuery(
                query="월차는 언제 받나요?",
                expected_sections=["월차", "1개월 만근", "당해년도 입사자"],
                description="월차 지급 조건 문의"
            ),
            TestQuery(
                query="AK복지몰은 어떻게 가입하나요?",
                expected_sections=["AK복지몰", "가입 방법", "회사 메일"],
                description="복지몰 가입 방법 문의"
            ),
            TestQuery(
                query="장기 근속자 연차 가산은 어떻게 되나요?",
                expected_sections=["장기 근속자", "가산일", "2년에 1일"],
                description="장기근속 연차 가산 문의"
            ),
            TestQuery(
                query="스낵바에는 뭐가 있나요?",
                expected_sections=["스낵바", "커피머신", "간식", "무료"],
                description="사내 스낵바 시설 문의"
            ),
            TestQuery(
                query="난임 휴가는 몇 일인가요?",
                expected_sections=["난임 휴가", "연 3일", "유급"],
                description="난임 휴가 일수 문의"
            )
        ]
    
    def setup_vectorstore(self):
        """벡터스토어 설정 및 문서 로드"""
        print("🚀 벡터스토어 설정 시작...")
        
        # OpenAI API 키 확인
        if not os.getenv("OPENAI_API_KEY"):
            raise ValueError("OPENAI_API_KEY가 .env 파일에 없습니다!")
        
        # 임베딩 모델 초기화
        self.embeddings = OpenAIEmbeddings(
            model="text-embedding-3-small",
            openai_api_key=os.getenv("OPENAI_API_KEY")
        )
        
        # 문서 로드
        documents = load_hr_document(self.file_path)
        print(f"📄 문서 로드 완료: {len(documents)}개 청크")
        
        # FAISS 벡터스토어 생성
        start_time = time.time()
        self.vectorstore = FAISS.from_documents(documents, self.embeddings)
        setup_time = time.time() - start_time
        
        print(f"✅ 벡터스토어 설정 완료 ({setup_time:.2f}초)")
        
    def _is_relevant_result(self, query_obj: TestQuery, retrieved_content: str) -> bool:
        """검색 결과가 관련성이 있는지 판단"""
        content_lower = retrieved_content.lower()
        
        # 예상 섹션 키워드 중 하나라도 포함되어 있으면 관련성 있음
        for section_keyword in query_obj.expected_sections:
            if section_keyword.lower() in content_lower:
                return True
        return False
    
    def search_and_evaluate(self, query_obj: TestQuery) -> EvaluationResult:
        """단일 쿼리에 대한 검색 및 평가"""
        
        # 유사도 검색 실행
        results = self.vectorstore.similarity_search_with_score(
            query_obj.query, 
            k=self.k
        )
        
        # 임계값 필터링
        filtered_results = [
            (doc, score) for doc, score in results 
            if score <= self.score_threshold
        ]
        
        # 결과 분석
        retrieved_sections = []
        scores = []
        is_relevant = []
        
        for doc, score in filtered_results:
            content = doc.page_content
            retrieved_sections.append(content[:100] + "..." if len(content) > 100 else content)
            scores.append(score)
            is_relevant.append(self._is_relevant_result(query_obj, content))
        
        # 메트릭 계산
        relevant_count = sum(is_relevant)
        total_retrieved = len(filtered_results)
        
        precision_at_k = relevant_count / total_retrieved if total_retrieved > 0 else 0
        hit_rate = relevant_count > 0
        
        return EvaluationResult(
            query=query_obj.query,
            precision_at_k=precision_at_k,
            hit_rate=hit_rate,
            retrieved_sections=retrieved_sections,
            scores=scores,
            is_relevant=is_relevant
        )
    
    def run_evaluation(self) -> Dict:
        """전체 평가 실행"""
        print("🎯 유사도 검색 정확도 평가 시작...")
        print(f"📊 설정: k={self.k}, threshold={self.score_threshold}")
        print("-" * 60)
        
        results = []
        total_precision = 0
        total_hit_rate = 0
        
        for i, query_obj in enumerate(self.test_queries, 1):
            print(f"[{i:2d}/{len(self.test_queries)}] {query_obj.query}")
            
            try:
                result = self.search_and_evaluate(query_obj)
                results.append(result)
                
                total_precision += result.precision_at_k
                total_hit_rate += int(result.hit_rate)
                
                # 실시간 결과 표시
                status = "✅ HIT" if result.hit_rate else "❌ MISS"
                print(f"         {status} | Precision: {result.precision_at_k:.2f} | Retrieved: {len(result.retrieved_sections)}")
                
            except Exception as e:
                print(f"         ❌ ERROR: {e}")
                
        # 전체 통계 계산
        avg_precision = total_precision / len(self.test_queries)
        avg_hit_rate = total_hit_rate / len(self.test_queries)
        
        print("-" * 60)
        print(f"📈 전체 결과:")
        print(f"   평균 Precision@{self.k}: {avg_precision:.3f}")
        print(f"   평균 Hit Rate@{self.k}: {avg_hit_rate:.3f}")
        print(f"   성공한 쿼리: {total_hit_rate}/{len(self.test_queries)}")
        
        return {
            'results': results,
            'avg_precision': avg_precision,
            'avg_hit_rate': avg_hit_rate,
            'total_queries': len(self.test_queries),
            'successful_queries': total_hit_rate
        }
    
    def generate_detailed_report(self, evaluation_results: Dict):
        """상세 결과 리포트 생성"""
        print("\n" + "="*80)
        print("📋 상세 평가 리포트")
        print("="*80)
        
        results = evaluation_results['results']
        
        print(f"\n🎯 전체 성능 요약")
        print(f"   총 테스트 쿼리: {evaluation_results['total_queries']}개")
        print(f"   평균 Precision@{self.k}: {evaluation_results['avg_precision']:.3f}")
        print(f"   평균 Hit Rate@{self.k}: {evaluation_results['avg_hit_rate']:.3f}")
        print(f"   검색 설정: k={self.k}, threshold={self.score_threshold}")
        
        # 성공/실패 케이스 분석
        successful_queries = [r for r in results if r.hit_rate]
        failed_queries = [r for r in results if not r.hit_rate]
        
        print(f"\n✅ 성공한 쿼리 ({len(successful_queries)}개):")
        for result in successful_queries:
            print(f"   • {result.query} (Precision: {result.precision_at_k:.2f})")
        
        if failed_queries:
            print(f"\n❌ 실패한 쿼리 ({len(failed_queries)}개):")
            for result in failed_queries:
                print(f"   • {result.query}")
                if result.retrieved_sections:
                    print(f"     검색된 내용: {result.retrieved_sections[0][:50]}...")
        
        # 상세 결과 (선택적 표시)
        print(f"\n📊 상세 검색 결과 (상위 5개만 표시):")
        for i, result in enumerate(results[:5], 1):
            print(f"\n[{i}] 쿼리: {result.query}")
            print(f"    Precision@{self.k}: {result.precision_at_k:.3f} | Hit Rate: {result.hit_rate}")
            
            for j, (section, score, relevant) in enumerate(zip(result.retrieved_sections, result.scores, result.is_relevant)):
                status = "✅" if relevant else "❌"
                print(f"    {j+1}. {status} (Score: {score:.3f}) {section}")
        
        # 개선 제안
        print(f"\n💡 개선 제안:")
        if evaluation_results['avg_precision'] < 0.7:
            print(f"   • Precision이 낮습니다. 임계값을 {self.score_threshold + 0.05:.2f}로 높여보세요.")
        if evaluation_results['avg_hit_rate'] < 0.8:
            print(f"   • Hit Rate가 낮습니다. k값을 {self.k + 2}로 늘리거나 임계값을 {self.score_threshold - 0.05:.2f}로 낮춰보세요.")
        if evaluation_results['avg_precision'] > 0.9 and evaluation_results['avg_hit_rate'] > 0.9:
            print(f"   • 성능이 우수합니다! 현재 설정을 유지하세요.")

def main():
    """메인 실행 함수"""
    # 설정
    file_path = "04_복지정책_v1.0.md"  # 실제 파일 경로로 변경
    k = 3
    score_threshold = 0.75
    
    # 평가기 초기화
    evaluator = HRSimilaritySearchEvaluator(file_path, k, score_threshold)
    
    try:
        # 벡터스토어 설정
        evaluator.setup_vectorstore()
        
        # 평가 실행
        results = evaluator.run_evaluation()
        
        # 상세 리포트 생성
        evaluator.generate_detailed_report(results)
        
    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        print("파일 경로와 환경 설정을 확인해주세요.")

if __name__ == "__main__":
    main()

🚀 벡터스토어 설정 시작...
📄 문서 로드 완료: 15개 청크
✅ 벡터스토어 설정 완료 (1.13초)
🎯 유사도 검색 정확도 평가 시작...
📊 설정: k=3, threshold=0.75
------------------------------------------------------------
[ 1/15] 연차휴가는 몇 일인가요?
         ❌ MISS | Precision: 0.00 | Retrieved: 0
[ 2/15] 병가 신청은 어떻게 하나요?
         ❌ MISS | Precision: 0.00 | Retrieved: 0
[ 3/15] 복지포인트는 얼마나 받나요?
         ❌ MISS | Precision: 0.00 | Retrieved: 0
[ 4/15] 교육비 지원 한도는 얼마인가요?
         ❌ MISS | Precision: 0.00 | Retrieved: 0
[ 5/15] 개인 장비 구입 지원은 어떻게 되나요?
         ❌ MISS | Precision: 0.00 | Retrieved: 0
[ 6/15] 건강검진은 누가 받을 수 있나요?
         ❌ MISS | Precision: 0.00 | Retrieved: 0
[ 7/15] 육아 지원금은 얼마인가요?
         ❌ MISS | Precision: 0.00 | Retrieved: 0
[ 8/15] 동아리 활동비는 어떻게 지원되나요?
         ❌ MISS | Precision: 0.00 | Retrieved: 0
[ 9/15] 가족돌봄휴가는 유급인가요?
         ❌ MISS | Precision: 0.00 | Retrieved: 0
[10/15] 태아 검진 휴가는 몇 번 사용할 수 있나요?
         ❌ MISS | Precision: 0.00 | Retrieved: 0
[11/15] 월차는 언제 받나요?
         ❌ MISS | Precision: 0.00 | Retrieved: 0
[12/15] AK복지몰은

In [1]:
# 라이브러리 import
import os
import numpy as np
from datetime import datetime
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from typing import List, Dict

# .env 파일에서 환경 변수 로드
# 노트북 파일과 같은 위치에 .env 파일이 있어야 합니다.
# .env 파일 내용: OPENAI_API_KEY="your_api_key"
load_dotenv()

True

In [3]:
def evaluate_retrieval(retriever, test_queries: List[Dict]) -> (float, float, List[Dict]):
    """
    검색 성능을 평가하고 결과를 반환합니다.
    """
    total_precision = 0
    total_hit_rate = 0
    results = []

    for query_info in test_queries:
        query = query_info["query"]
        expected_sections = query_info["expected_sections"]
        
        # 유사도 검색 실행
        retrieved_docs = retriever.invoke(query)
        
        # 검색된 문서의 'Header 2' 메타데이터를 추출합니다.
        retrieved_sections = [doc.metadata.get('Header 2', 'N/A') for doc in retrieved_docs]
        
        # Precision@K 계산: 검색된 K개 중 정답의 비율
        correct_predictions = sum(1 for section in retrieved_sections if section in expected_sections)
        precision = correct_predictions / len(retrieved_docs) if retrieved_docs else 0
        total_precision += precision
        
        # Hit Rate@K 계산: 검색된 K개 중 정답이 하나라도 포함되었는지 여부
        hit = 1 if any(section in expected_sections for section in retrieved_sections) else 0
        total_hit_rate += hit
        
        results.append({
            "query": query,
            "expected": expected_sections,
            "retrieved": retrieved_sections,
            "precision": precision,
            "hit": hit
        })
        
    avg_precision = total_precision / len(test_queries) if test_queries else 0
    avg_hit_rate = total_hit_rate / len(test_queries) if test_queries else 0
    
    return avg_precision, avg_hit_rate, results

def generate_report(avg_precision: float, avg_hit_rate: float, results: List[Dict], k: int, threshold: float) -> str:
    """
    평가 결과를 바탕으로 리포트 문자열을 생성합니다.
    """
    report = []
    report.append(f"# 검색 정확도 평가 리포트 ({datetime.now().strftime('%Y-%m-%d %H:%M:%S')})")
    report.append("\n## 1. 평가 설정")
    report.append(f"- 임베딩 모델: `text-embedding-3-small`")
    report.append(f"- 검색 파라미터: Top-K={k}, 유사도 임계값={threshold}")
    
    report.append("\n## 2. 종합 평가 결과")
    report.append(f"- **Precision@{k}: {avg_precision:.2%}**")
    report.append(f"- **Hit Rate@{k}: {avg_hit_rate:.2%}**")
    
    report.append("\n## 3. 개별 쿼리 결과")
    for res in results:
        report.append(f"\n### 쿼리: \"{res['query']}\"")
        report.append(f"- **예상 정답 섹션**: {res['expected']}")
        report.append(f"- **검색된 섹션**: {res['retrieved']}")
        report.append(f"- **Precision**: {res['precision']:.2f}")
        report.append(f"- **Hit**: {'Yes' if res['hit'] else 'No'}")
        
    return "\n".join(report)


In [4]:
# --- 평가 실행 ---

# 1. 문서 로드 (노트북에 이미 정의된 함수 사용)
# 파일 경로는 실제 파일 위치에 맞게 수정해주세요.
file_path = "04_복지정책_v1.0.md"
docs = load_hr_document(file_path)

# 2. 임베딩 및 벡터스토어 생성
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vectorstore = FAISS.from_documents(docs, embeddings)

# 3. 검색 파라미터 설정
K = 3
SCORE_THRESHOLD = 0.5
retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={'k': K, 'score_threshold': SCORE_THRESHOLD}
)

# 4. 테스트 쿼리 및 예상 정답 정의
# 'expected_sections'는 load_hr_document 함수가 생성하는 메타데이터('Header 2')와 일치해야 합니다.
test_queries = [
    {"query": "연차는 며칠 주나요?", "expected_sections": ["1. 휴가 및 휴직 제도"]},
    {"query": "병가 사용법 알려줘", "expected_sections": ["1. 휴가 및 휴직 제도"]},
    {"query": "복지포인트는 얼마야?", "expected_sections": ["2. 복지포인트 및 지원금"]},
    {"query": "교육비 지원 받을 수 있어?", "expected_sections": ["2. 복지포인트 및 지원금"]},
    {"query": "노트북 말고 개인 장비도 사주나요?", "expected_sections": ["3. 업무 장비 지원"]},
    {"query": "건강검진은 누가 받을 수 있어?", "expected_sections": ["4. 종합 건강검진"]},
    {"query": "동아리 만들고 싶어", "expected_sections": ["6. 사내 동아리 활동 지원"]},
    {"query": "육아 지원금 신청 어떻게 해?", "expected_sections": ["7. 임신·출산·육아 지원"]},
]

# 5. 평가 실행
avg_precision, avg_hit_rate, results = evaluate_retrieval(retriever, test_queries)

# 6. 리포트 생성 및 출력
report_content = generate_report(avg_precision, avg_hit_rate, results, K, SCORE_THRESHOLD)

report_filename = f"retrieval_evaluation_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.md"
with open(report_filename, "w", encoding="utf-8") as f:
    f.write(report_content)
    
print(f"평가 완료! '{report_filename}' 파일에 리포트가 저장되었습니다.")
print("\n--- 리포트 미리보기 ---")
# 노트북 환경에서는 Markdown을 더 예쁘게 출력할 수 있습니다.
from IPython.display import display, Markdown
display(Markdown(report_content))


No relevant docs were retrieved using the relevance score threshold 0.5
No relevant docs were retrieved using the relevance score threshold 0.5
c:\Users\AY\Desktop\PROJECTS\HR-chatbot-development-project\venv\Lib\site-packages\langchain_core\vectorstores\base.py:1070: UserWarning: Relevance scores must be between 0 and 1, got [(Document(id='b703f21c-94bc-4f45-9f71-49adc26e0cfd', metadata={'문서제목': '가이다 플레이 스튜디오(GPS) 직원 복지제도 종합 안내서', '정책대분류': '2. 복지포인트 및 지원금', '정책세부항목': '2.1 복지포인트'}, page_content='## 2. 복지포인트 및 지원금  \n### 2.1 복지포인트\n- 연간 300만원의 복지포인트가 지급됩니다.\n- 당해년도 입사자(1년 미만 근속자)는 근무 개월 수에 비례하여 월할 계산하여 당해분 복지포인트가 지급됩니다. (지급일: 수습기간 종료일)\n- AK복지몰(https://akfamily.com)에서만 사용 가능합니다.\n- 포인트는 매년 1월에 일괄 지급됩니다.\n#### **가입 방법**\n1. AK복지몰(https://akfamily.com)에 접속하여 회원가입 클릭\n2. 소속에 "가이다 플레이 스튜디오" 입력\n3. 회사 메일주소 입력 후 이메일 인증\n#### **신청 절차**\n- 별도 신청 절차 없이 사용 가능합니다.\n- 사용 내역은 AK복지몰 사이트에서 확인 가능합니다.\n---'), np.float32(0.24179387)), (Document(id='bee8c991-3d3e-45a9-ae7f-f8c39aeeddc4', metadata={'문서제목':

평가 완료! 'retrieval_evaluation_report_20250922_210059.md' 파일에 리포트가 저장되었습니다.

--- 리포트 미리보기 ---


# 검색 정확도 평가 리포트 (2025-09-22 21:00:59)

## 1. 평가 설정
- 임베딩 모델: `text-embedding-3-small`
- 검색 파라미터: Top-K=3, 유사도 임계값=0.5

## 2. 종합 평가 결과
- **Precision@3: 0.00%**
- **Hit Rate@3: 0.00%**

## 3. 개별 쿼리 결과

### 쿼리: "연차는 며칠 주나요?"
- **예상 정답 섹션**: ['1. 휴가 및 휴직 제도']
- **검색된 섹션**: []
- **Precision**: 0.00
- **Hit**: No

### 쿼리: "병가 사용법 알려줘"
- **예상 정답 섹션**: ['1. 휴가 및 휴직 제도']
- **검색된 섹션**: []
- **Precision**: 0.00
- **Hit**: No

### 쿼리: "복지포인트는 얼마야?"
- **예상 정답 섹션**: ['2. 복지포인트 및 지원금']
- **검색된 섹션**: []
- **Precision**: 0.00
- **Hit**: No

### 쿼리: "교육비 지원 받을 수 있어?"
- **예상 정답 섹션**: ['2. 복지포인트 및 지원금']
- **검색된 섹션**: []
- **Precision**: 0.00
- **Hit**: No

### 쿼리: "노트북 말고 개인 장비도 사주나요?"
- **예상 정답 섹션**: ['3. 업무 장비 지원']
- **검색된 섹션**: []
- **Precision**: 0.00
- **Hit**: No

### 쿼리: "건강검진은 누가 받을 수 있어?"
- **예상 정답 섹션**: ['4. 종합 건강검진']
- **검색된 섹션**: []
- **Precision**: 0.00
- **Hit**: No

### 쿼리: "동아리 만들고 싶어"
- **예상 정답 섹션**: ['6. 사내 동아리 활동 지원']
- **검색된 섹션**: []
- **Precision**: 0.00
- **Hit**: No

### 쿼리: "육아 지원금 신청 어떻게 해?"
- **예상 정답 섹션**: ['7. 임신·출산·육아 지원']
- **검색된 섹션**: []
- **Precision**: 0.00
- **Hit**: No